In [ ]:
# Fine tuining using BERT model from huggingface (bert-base-uncased)

!pip install torch transformers peft datasets #used to access we hugging face model & parameter we use transformers libraries


In [ ]:
from datasets import load_dataset
dataset = load_dataset("SetFit/emotion")  # here we are using SetFit/emotion data set to fine tune the model

README.md:   0%|          | 0.00/194 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/2.23M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/276k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/279k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# from the above model we have train and test data here we are just extracting that DATA SETS

train_data = dataset["train"]
test_data = dataset["test"]


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# importing our tokenizer and using BERT MODEL


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Tokinizing our trained and test data using funcn

# Tokenization function
def tokenize_function(batch):
  return tokenizer(batch["text"], padding="max_length", truncation=True)

# Apply tokenization to train and test data
train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)

# Convert to PyTorch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# Perfoming sequence classification

from transformers import AutoModelForSequenceClassification

# Load BERT model(6 output classes)
base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# Define LoRA configuration      ***HERE WE USE LORA MODEL***
lora_config = LoraConfig(
    r=8, # Low rank adaptation dimension
    lora_alpha = 32, # scaling factor
    lora_dropout = 0.05, # Dropout rate
    target_modules = ["query", "value"] # Apply LORA to self-attention layers only
)

# prepare model for LORA
base_model = prepare_model_for_kbit_training(base_model)

# convert model into LORA-enabled model
peft_model = get_peft_model(base_model, lora_config)

# Print trainable parameters
peft_model.print_trainable_parameters()


# NOTE : all params: 109,781,766 this nor is the pre-trained model which is 100million
       # by using BERT now we are just training the (rainable params: 294,912) which is 0.2%
       # from the trained model we saved 99% of the computaion power and we are fine-tuining
       # our model accroding to our model needs


trainable params: 294,912 || all params: 109,781,766 || trainable%: 0.2686


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments( # Configuration class that defines how training should happen
    output_dir = "./model_checkpoints", # where to save model
    num_train_epochs = 3, # Train for 3 epochs
    per_device_train_batch_size =16, # 16 samples per GPU/CPU
    eval_strategy = "epoch", # Evaluate after every epoch
    save_strategy = "epoch", # save model after each epoch
    logging_steps = 10, # Log training metrics every 10 steps
    load_best_model_at_end = True, # Automatically load best checkpoint
    fp16=True # Use mixed precision for faster training (if GPU supports it)

)

# A high-level class that automates training, evaluation, and saving models.
# It wraps around ur model and dataset, handling:

# Training loops.
# Evaluation during training.
# Model saving & checkpointing.

trainer = Trainer(
    model = peft_model,  # LORA fine-tuned model
    args = training_args, # Training settings
    train_dataset = train_dataset, # Training data
    eval_dataset = test_dataset,  # Test data
    tokenizer = tokenizer # Tokenizer for processing text
)


trainer.train()

/tmp/ipython-input-3410886032.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.261300,1.129268
2,1.044300,0.981926


Epoch,Training Loss,Validation Loss
1,1.261300,1.129268
2,1.044300,0.981926
3,1.045600,0.926200


TrainOutput(global_step=3000, training_loss=1.1901638793945313, metrics={'train_runtime': 1055.1867, 'train_samples_per_second': 45.49, 'train_steps_per_second': 2.843, 'total_flos': 1.2673270775808e+16, 'train_loss': 1.1901638793945313, 'epoch': 3.0})

In [ ]:
from transformers import BitsAndBytesConfig


#         *** Q-LORA MODEL ***

# Apply 4-bit quantization to reduce memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True, # Enable 4-bit quantization
    bnb_4bit_compute_dtype = torch.float16, # Compute in float16
    bnb_4bit_use_double_quant = True, # Use double quantization for efficiency
)


# Load quantized BERT model
base_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=6,
    quantization_config=bnb_config
)

# Apply LORA On top of QLROA
peft_model = get_peft_model(base_model, lora_config)

PackageNotFoundError: No package metadata was found for bitsandbytes